In [ ]:
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.6/autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl

In [2]:
!pip install einops autoawq==0.1.7

  Obtaining dependency information for transformers>=4.35.0 from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.0
    Uninstalling transformers-4.34.0:
      Successfully uninstalled transformers-4.34.0


In [ ]:
!huggingface-cli login --token <MY_HF_TOKEN>

PhoGPT-7B5 QUANTIZATION

In [3]:
pip show transformers torch autoawq

Name: transformers
Version: 4.35.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: autoawq, lm-eval
---
Name: torch
Version: 2.1.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, autoawq, lm-eval, torchaudio, torchelastic, torchvision
---

In [4]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'vinai/PhoGPT-7B5-Instruct'
quant_path = 'PhoGPT-7B5-Instruct-AWQ'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, trust_remote_code=True,device_map={"": "cpu"}, **{"low_cpu_mem_usage": True})
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)


/home/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-7B5-Instruct/d1a5a418bf01d49e8bf1b5b737b8ef143a33d9fd/configuration_mpt.py:97: UserWarning: alibi is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi is turned on, setting `learned_pos_emb` to `False.`')


Fetching 24 files:   0%|          | 0/24 [00:00<?, ?it/s]

/home/.cache/huggingface/modules/transformers_modules/d1a5a418bf01d49e8bf1b5b737b8ef143a33d9fd/configuration_mpt.py:97: UserWarning: alibi is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi is turned on, setting `learned_pos_emb` to `False.`')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.
/opt/conda/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
Token indices sequence length is longer than the specified maximum sequence length for this model (7822 > 2048). Running this sequence through the model will result in indexing errors
AWQ:   0%|          | 0/32 [00:02<?, ?it/s]


TypeError: GroupedQueryAttention.forward() got an unexpected keyword argument 'output_attentions'

In [ ]:
# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

In [ ]:
model_name = "PhoGPT-7B5-Instruct-AWQ"
HUGGING_FACE_USER_NAME="phatjk"
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)
tokenizer.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

vietcuna-7b-v3 QUANTIZATION

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'vilm/vietcuna-7b-v3'
quant_path = 'vietcuna-7b-v3-AWQ'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path,device_map={"": "cpu"}, trust_remote_code=True, **{"low_cpu_mem_usage": True})
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.
/opt/conda/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
AWQ: 100%|██████████| 30/30 [19:33<00:00, 39.12s/it]


In [2]:
# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('vietcuna-7b-v3-AWQ/tokenizer_config.json',
 'vietcuna-7b-v3-AWQ/special_tokens_map.json',
 'vietcuna-7b-v3-AWQ/tokenizer.json')

In [6]:
from huggingface_hub import HfApi
api = HfApi()
model_name = "vietcuna-7b-v3-AWQ"
HUGGING_FACE_USER_NAME="phatjk"
api.upload_folder(
    folder_path="vietcuna-7b-v3-AWQ",
    repo_id=HUGGING_FACE_USER_NAME+"/"+model_name,
    repo_type="model",
)

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/phatjk/vietcuna-7b-v3-AWQ/tree/main/'